In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_curve, auc, precision_recall_curve

import pubchempy as pcp
import re
import requests
from IPython.display import Image
import json
import time
import requests
import ast

from rxnmapper import RXNMapper
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import Draw
from rdkit.Chem import rdChemReactions
from rdkit.Chem.Draw import rdMolDraw2D
from rdkit.Chem.Draw import IPythonConsole
from rdkit import DataStructs
from IPython.display import SVG, display, Image

In [ ]:
np.random.seed(42)

def accuracy(actual, predicted):
    return np.mean(actual == predicted)

valid = pd.read_csv('valid_fingerprints_atommap.csv')
invalid_from_fingerprint = pd.read_csv('invalid_from_fingerprint_fingerprints_atommap.csv')
invalid_from_atommap = pd.read_csv("invalid_from_atommap_fingerprints_atommap.csv")

mapper = 'rm'

valid_from_fingerprint = valid.drop(['Unnamed: 0', 'substrates_converted_smiles', 'products_converted_smiles','rxnmapper_mapped_smiles',
       'MorganFingerprint_similarity', 
       'AtomPairFingerprint_similarity', 
       'MACCSKeysFingerprint_similarity'], axis=1)
valid_from_atommap = valid.drop(['Unnamed: 0', 'substrates_converted_smiles', 'products_converted_smiles','rxnmapper_mapped_smiles', 
       f'{mapper}_prop_c_conserved_neighbors',
       f'{mapper}_prop_gained_c', 
       f'{mapper}_prop_lost_c',
       f'{mapper}_prop_unconserved_c_bonds',
       f'{mapper}_prop_unmapped_c',
       f'{mapper}_prop_unmapped_c_bonds', 
       f'{mapper}_prop_map_to_unmap_c_bonds', 
       f'{mapper}_prop_unmap_to_unmap_c_bonds', 
       f'{mapper}_total_c', 
       f'{mapper}_total_unmap_c', 
       f'{mapper}_total_bonds'], axis=1)

invalid_from_fingerprint = invalid_from_fingerprint.drop(['Unnamed: 0', 'substrate_Cs', 'products_Cs', 'substrates_converted_smiles', 'products_converted_smiles','rxnmapper_mapped_smiles',
       'MorganFingerprint_similarity', 
       'AtomPairFingerprint_similarity', 
       'MACCSKeysFingerprint_similarity'], axis=1)
invalid_from_atommap = invalid_from_atommap.drop(['Unnamed: 0', 'substrate_Cs', 'products_Cs', 'substrates_converted_smiles', 'products_converted_smiles','rxnmapper_mapped_smiles', 
       f'{mapper}_prop_c_conserved_neighbors',
       f'{mapper}_prop_gained_c', 
       f'{mapper}_prop_lost_c',
       f'{mapper}_prop_unconserved_c_bonds',
       f'{mapper}_prop_unmapped_c',
       f'{mapper}_prop_unmapped_c_bonds', 
       f'{mapper}_prop_map_to_unmap_c_bonds', 
       f'{mapper}_prop_unmap_to_unmap_c_bonds', 
       f'{mapper}_total_c', 
       f'{mapper}_total_unmap_c', 
       f'{mapper}_total_bonds'], axis=1)

In [49]:
mapper = 'rm'
columns_from_atommap = [
       'MorganFingerprint_similarity', 
       'AtomPairFingerprint_similarity', 
       'MACCSKeysFingerprint_similarity']
feature_names_from_atommap = [
       'Morgan Fingerprint Similarity', 
       'Atom Pair Fingerprint Similarity', 
       'MACCS Key Fingerprint Similarity']

columns_from_fingerprint = [
       f'{mapper}_prop_c_conserved_neighbors',
       f'{mapper}_prop_gained_c', 
       f'{mapper}_prop_lost_c',
       f'{mapper}_prop_unconserved_c_bonds',
       f'{mapper}_prop_unmapped_c',
       f'{mapper}_prop_unmapped_c_bonds', 
       f'{mapper}_prop_map_to_unmap_c_bonds', 
       f'{mapper}_prop_unmap_to_unmap_c_bonds', 
       f'{mapper}_total_c', 
       f'{mapper}_total_unmap_c', 
       f'{mapper}_total_bonds']
feature_names_from_fingerprint = [
       f'Proportion of C atoms with conserved neighbors',
       f'Proportion of gained C atoms', 
       f'Proportion of lost C atoms',
       f'Proportion of unconserved C-to-C bonds',
       f'Proportion of unmapped C atoms',
       f'Proportion of unmapped C-to-C bonds', 
       f'Proportion of mapped C to unmapped C bonds', 
       f'Proportion of unmapped C to unmapped C bonds', 
       f'Total number of carbon atoms', 
       f'Total number of unmapped carbon atoms', 
       f'Total number of bonds']

In [ ]:
valid_from_fingerprint['label'] = 1
valid_from_atommap['label'] = 1
invalid_from_fingerprint['label'] = 0
invalid_from_atommap['label'] = 0

combined_from_fingerprint = pd.concat([valid_from_fingerprint, invalid_from_fingerprint], ignore_index=True)
combined_from_atommap = pd.concat([valid_from_atommap, invalid_from_atommap], ignore_index=True)

shuffle_from_fingerprint = shuffle(combined_from_fingerprint)
shuffle_from_atommap = shuffle(combined_from_atommap)

X_from_fingerprint = shuffle_from_fingerprint.drop('label', axis=1)
y_from_fingerprint = shuffle_from_fingerprint['label']

X_from_atommap = shuffle_from_atommap.drop('label', axis=1)
y_from_atommap = shuffle_from_atommap['label']

print(X_from_fingerprint.shape, y_from_fingerprint.shape, X_from_atommap.shape, y_from_atommap.shape)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_from_fingerprint, y_from_fingerprint, test_size=0.2, random_state=42)

X_train = X_train.dropna()
X_test = X_test.dropna()
y_train = y_train.loc[X_train.index]
y_test = y_test.loc[X_test.index]

rfc_for_atommap = RandomForestClassifier(n_estimators=300, random_state=42)
rfc_for_atommap.fit(X_train, y_train)
y_pred = rfc_for_atommap.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print("Classification metrics after feature selection:")
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

importances = rfc_for_atommap.feature_importances_

feature_importances = pd.DataFrame({'Feature': feature_names_from_fingerprint, 'Importance': importances})
feature_importances = feature_importances.sort_values(by='Importance', ascending=False)
print(feature_importances)

plt.figure(figsize=(7, 8))
plt.barh(
    feature_importances['Feature'][::-1],  
    feature_importances['Importance'][::-1],  
    color='blue',  
    alpha=0.6,  
    edgecolor='black'  
)
plt.xlabel('Importance Score', fontsize=9)
plt.ylabel('Feature', fontsize=9)
plt.yticks(fontsize=9)
plt.title(f'Top Feature Importances for Invalid Set Generated from Molecular Fingerprint Scores', fontsize=10)
plt.tight_layout()
plt.savefig('Top Feature Importances for Invalid Set Generated from Molecular Fingerprint Scores.png', bbox_inches='tight')
plt.show()

y_pred = rfc_for_atommap.predict(X_test)
y_proba = rfc_for_atommap.predict_proba(X_test)[:, 1]

# Compute PR and ROC curves
fpr, tpr, _ = roc_curve(y_test, y_proba)
roc_auc = auc(fpr, tpr)

precision, recall, _ = precision_recall_curve(y_test, y_proba)
pr_auc = auc(recall, precision)

# Plot ROC Curve
plt.figure(figsize=(5, 5))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=1, linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve with Invalid Set Generated from Molecular Fingerprint Scores')
plt.legend(loc='lower right')
plt.tight_layout()
plt.savefig('ROC Curve with Invalid Set Generated from Molecular Fingerprint Scores.png', bbox_inches='tight')
plt.show()

# Plot Precision-Recall Curve
plt.figure(figsize=(5, 5))
plt.plot(recall, precision, color='blue', lw=2, label=f'PR curve (AUC = {pr_auc:.2f})')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve with Invalid Set Generated from Molecular Fingerprint Scores')
plt.legend(loc='upper right')
plt.tight_layout()
plt.savefig('Precision-Recall Curve with Invalid Set Generated from Molecular Fingerprint Scores.png', bbox_inches='tight')
plt.show()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_from_atommap, y_from_atommap, test_size=0.2, random_state=42)

X_train = X_train.dropna()
X_test = X_test.dropna()
y_train = y_train.loc[X_train.index]
y_test = y_test.loc[X_test.index]

rfc_for_fingerprint = RandomForestClassifier(n_estimators=300, random_state=42)
rfc_for_fingerprint.fit(X_train, y_train)
y_pred = rfc_for_fingerprint.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print("Classification metrics after feature selection:")
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

importances = rfc_for_fingerprint.feature_importances_

feature_importances = pd.DataFrame({'Feature': feature_names_from_atommap, 'Importance': importances})
feature_importances = feature_importances.sort_values(by='Importance', ascending=False)
print(feature_importances)

plt.figure(figsize=(7, 4))
plt.barh(
    feature_importances['Feature'][::-1],  
    feature_importances['Importance'][::-1],  
    color='blue',  
    alpha=0.6,  
    edgecolor='black'  
)
plt.xlabel('Importance Score', fontsize=9)
plt.ylabel('Feature', fontsize=9)
plt.yticks(fontsize=9)
plt.title(f'Top Feature Importances for Invalid Set Generated from Atom Mapping Metrics', fontsize=10)
plt.tight_layout()
plt.savefig('Top Feature Importances for Invalid Set Generated from Atom Mapping Metrics.png', bbox_inches='tight')
plt.show()

y_pred = rfc_for_fingerprint.predict(X_test)
y_proba = rfc_for_fingerprint.predict_proba(X_test)[:, 1]

# Compute PR and ROC curves
fpr, tpr, _ = roc_curve(y_test, y_proba)
roc_auc = auc(fpr, tpr)

precision, recall, _ = precision_recall_curve(y_test, y_proba)
pr_auc = auc(recall, precision)

# Plot ROC Curve
plt.figure(figsize=(5, 5))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=1, linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve with Invalid Set Generated from Atom Mapping Metrics')
plt.legend(loc='lower right')
plt.tight_layout()
plt.savefig('ROC Curve with Invalid Set Generated from Atom Mapping Metrics.png', bbox_inches='tight')
plt.show()

# Plot Precision-Recall Curve
plt.figure(figsize=(5, 5))
plt.plot(recall, precision, color='blue', lw=2, label=f'PR curve (AUC = {pr_auc:.2f})')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve with Invalid Set Generated from Atom Mapping Metrics')
plt.legend(loc='upper right')
plt.tight_layout()
plt.savefig('Precision-Recall Curve with Invalid Set Generated from Atom Mapping Metrics.png', bbox_inches='tight')
plt.show()

In [ ]:
gpt = pd.read_csv("gpt_extracted.csv")
gpt['substrates'] = gpt['substrates'].apply(ast.literal_eval)
gpt['products'] = gpt['products'].apply(ast.literal_eval)

In [ ]:
pubchem_matches = set()
cactus_matches = set()
queried = set()
pubchem_no_match = set()
cactus_no_match = set()
no_matches = set()

def get_smiles_pubchem(name):
  smiles = pcp.get_compounds(name, 'name')

  if smiles:
    return smiles[0].isomeric_smiles

  else:
    return None

def get_smiles_cactus(structure_identifier):
    url = f"https://cactus.nci.nih.gov/chemical/structure/{structure_identifier}/smiles"

    response = requests.get(url)

    if response.status_code == 200:
        return response.text
    else:
        return None
            
def get_smiles_chemspider(name):
    api_key = ''

    search_url = f'https://api.rsc.org/compounds/v1/filter/name/'
    headers = {'apikey': api_key}

    try:
        data = {'name': name}

        json_data = json.dumps(data)
        response = requests.post(search_url, headers=headers, data=json_data)
        response.raise_for_status()  
        data = response.json()

        csid = data['queryId']

        # Give the server time to load the query 
        time.sleep(1)

        details_url = f'https://api.rsc.org/compounds/v1/filter/{csid}/results'
        response = requests.get(details_url, headers=headers)
        response.raise_for_status()
        data = response.json()

        results = data['results'][0]
        details_url = f'https://api.rsc.org/compounds/v1/records/{results}/details?fields=SMILES'
        response = requests.get(details_url, headers=headers)
        response.raise_for_status()  
        data = response.json()

        print(data['smiles'])
        return data['smiles']

    except Exception as e:
        print(f'Error: {e}')
        print(f"chemspider: no match for {name}")
        return None

def convert_smiles_to_structure(smiles):
  molecule = Chem.MolFromSmiles(smiles)

  if molecule is not None:
      img = Draw.MolToImage(molecule)
      img.save("molecule.png")
      display(Image(filename="molecule.png"))
      print("SMILES notation is valid.")
  else:
      print("Invalid SMILES notation.")

def smiles_tokenizer(smiles):
    """
    Tokenize a SMILES molecule or reaction
    """
    pattern =  "(\[[^\]]+]|Br?|Cl?|N|O|S|P|F|I|b|c|n|o|s|p|\(|\)|\.|=|#|-|\+|\\\\|\/|:|~|@|\?|>|\*|\$|\%[0-9]{2}|[0-9])"
    regex = re.compile(pattern)
    tokens = [token for token in regex.findall(smiles)]
    assert smiles == ''.join(tokens)
    return ' '.join(tokens)

def get_smiles(mols, tried=False):
    for mol in mols:
        if mol != '?':
            pubchem_smile = None
            cactus_smile = None

            if mol not in queried:
                try:
                    pubchem_smile = get_smiles_pubchem(mol)
                    if pubchem_smile:
                        pubchem_matches.add((mol, pubchem_smile))
                    else:
                        pubchem_no_match.add(mol)
                        print(f"pubchem: no match for {mol}")
                except:
                    pubchem_no_match.add(mol)
                    print("pubchem: an error occurred")

                try:
                    cactus_smile = get_smiles_cactus(mol)
                    if cactus_smile:
                        cactus_matches.add((mol, cactus_smile))
                    else:
                        cactus_no_match.add(mol)
                        print(f"cactus: no match for {mol}")
                except:
                    cactus_no_match.add(mol)
                    print("cactus: an error occurred")
                    
                if not pubchem_smile and not cactus_smile:
                    no_matches.add(mol)
                    
                queried.add(mol)
            

def process_chemical_column_gpt(column_str):
    smiles = []
    
    for chem in column_str:
        get_smiles([chem])
        smile = next(
            (s for m, s in pubchem_matches if m == chem),
            next(
                (s for m, s in cactus_matches if m == chem),
                None
            )
        )
        
        if smile:
            smiles.append(smile)
    
    return smiles

In [ ]:
def process_reaction_dataframes(gpt):
    df_processed = gpt.copy()
    
    brenda_substrates_results = df_processed['substrates'].apply(process_chemical_column_gpt)
    brenda_products_results = df_processed['products'].apply(process_chemical_column_gpt)
    df_processed['substrates_converted_smiles'] = brenda_substrates_results
    df_processed['products_converted_smiles'] = brenda_products_results
    
    return df_processed

df_processed = process_reaction_dataframes(gpt)

In [ ]:
gpt = df_processed[['substrates', 'products', 'substrates_converted_smiles','products_converted_smiles']]

# Remove common cofactors
for index, row in gpt.iterrows():
    substrate = row['substrates_converted_smiles']
    pdt = row['products_converted_smiles']
    
    if isinstance(substrate, str):
        substrate = ast.literal_eval(substrate)
    if isinstance(pdt, str):
        pdt = ast.literal_eval(pdt)
    
    nad_plus = 'C1=CC(=C[N+](=C1)[C@H]2[C@@H]([C@@H]([C@H](O2)COP(=O)([O-])OP(=O)(O)OC[C@@H]3[C@H]([C@H]([C@@H](O3)N4C=NC5=C(N=CN=C54)N)O)O)O)O)C(=O)N'
    nadh = 'C1C=CN(C=C1C(=O)N)[C@H]2[C@@H]([C@@H]([C@H](O2)COP(=O)(O)OP(=O)(O)OC[C@@H]3[C@H]([C@H]([C@@H](O3)N4C=NC5=C(N=CN=C54)N)O)O)O)O'
    nadp_plus = 'C1=CC(=C[N+](=C1)[C@H]2[C@@H]([C@@H]([C@H](O2)COP(=O)(O)OP(=O)(O)OC[C@@H]3[C@H]([C@H]([C@@H](O3)N4C=NC5=C(N=CN=C54)N)OP(=O)(O)O)O)O)O)C(=O)N'
    nadph = 'C1C=CN(C=C1C(=O)N)[C@H]2[C@@H]([C@@H]([C@H](O2)COP(=O)(O)OP(=O)(O)OC[C@@H]3[C@H]([C@H]([C@@H](O3)N4C=NC5=C(N=CN=C54)N)OP(=O)(O)O)O)O)O'
    hydrogen = '[H+]'

    cofactors = [nad_plus, nadh, nadp_plus, nadph, hydrogen]

    for i in cofactors:
        if i in substrate:
            substrate.remove(i)
        if i in pdt:
            pdt.remove(i)

    if len(substrate) == 0 or len(pdt) == 0:
        continue
    
gpt = gpt[gpt['substrates_converted_smiles'].astype(bool) & gpt['products_converted_smiles'].astype(bool)]

In [ ]:
rxn_mapper = RXNMapper()

def rxnmapper_map_reaction(reactants_smiles, products_smiles):
    reactants_str = '.'.join(reactants_smiles)
    products_str = '.'.join(products_smiles)

    reaction_smiles = f"{reactants_str}>>{products_str}"

    mapped_reaction = rxn_mapper.get_attention_guided_atom_maps([reaction_smiles])

    return mapped_reaction[0]['mapped_rxn']

def draw_chemical_reaction(smiles, highlightByReactant=False, font_scale=0.8):
    rxn = rdChemReactions.ReactionFromSmarts(smiles,useSmiles=True)
    trxn = rdChemReactions.ChemicalReaction(rxn)
    for m in trxn.GetReactants():
        moveAtomMapsToNotes(m)
    for m in trxn.GetProducts():
        moveAtomMapsToNotes(m)
    d2d = rdMolDraw2D.MolDraw2DSVG(900,300)
    d2d.drawOptions().annotationFontScale=font_scale
    d2d.DrawReaction(trxn,highlightByReactant=highlightByReactant)

    d2d.FinishDrawing()

    return d2d.GetDrawingText()

def moveAtomMapsToNotes(m):
    for at in m.GetAtoms():
        if at.GetAtomMapNum():
            at.SetProp("atomNote",str(at.GetAtomMapNum()))
            
def add_rxnmapper_mapping(df):
    mapped_rxn_smiles = []
    for i in range(df.shape[0]):
       mapped_rxn_smiles.append(rxnmapper_map_reaction(df['substrates_converted_smiles'].iloc[i], df['products_converted_smiles'].iloc[i]))
    df["rxnmapper_mapped_smiles"] = mapped_rxn_smiles
    
add_rxnmapper_mapping(gpt)

In [ ]:
def carbonMetrics(mapped_reaction_smiles):
    if(mapped_reaction_smiles == ">>"):
        return(-1,-1,-1,-1,-1,-1,-1,-1,-1,-1)
    
    # Pull reactions from smiles
    rdkit_reaction = AllChem.ReactionFromSmarts(mapped_reaction_smiles, useSmiles=True)

    reactant_labels = {}
    product_labels = {}

    for mol in rdkit_reaction.GetReactants():
        AllChem.Compute2DCoords(mol)

    for mol in rdkit_reaction.GetProducts():
        AllChem.Compute2DCoords(mol)

    for reactant in rdkit_reaction.GetReactants():
        for atomR in reactant.GetAtoms():
            label = f'{atomR.GetSymbol()}{atomR.GetAtomMapNum()}'
            reactant_labels[label] = atomR.GetSymbol()

    for product in rdkit_reaction.GetProducts():
        for atom in product.GetAtoms():
            label = f'{atom.GetSymbol()}{atom.GetAtomMapNum()}'
            product_labels[label] = atom.GetSymbol()

    # Total number of carbon atoms, proportion of carbon atoms gained or lost
    rdkit_reaction = AllChem.ReactionFromSmarts(mapped_reaction_smiles, useSmiles=True)
    CarbonR, CarbonP = 0,0
    CarbonsGained, CarbonsLost = 0,0

    # Count carbons in reactants
    for reactant in rdkit_reaction.GetReactants():
      for atomR in reactant.GetAtoms():
         if atomR.GetSymbol() == 'C':
          CarbonR += 1

    # Count carbons in products
    for product in rdkit_reaction.GetProducts():
      for atomP in product.GetAtoms():
         if atomP.GetSymbol() == 'C':
          CarbonP += 1

    # Count if carbons have been lost or gained
    if CarbonP > CarbonR:
      CarbonsGained = CarbonP-CarbonR
    elif CarbonR > CarbonP:
      CarbonsLost = CarbonR-CarbonP
    else:
      CarbonsGained = 0
      CarbonsLost = 0

    TotalCarbons = CarbonP + CarbonR
    if TotalCarbons == 0:
      TotalCarbons = -1
    propCarbonsGained = CarbonsGained/TotalCarbons
    propCarbonsLost = CarbonsLost/TotalCarbons
    CarbonMetrics = [TotalCarbons,propCarbonsGained,propCarbonsLost]

    # total number of C-C bonds, proportion of unmapped carbon atoms, proportion of bonds involving at least one unmapped carbon
    # proportion of mapped-unmapped bonds, proportion of unmapped-unmapped bonds, total number of unmapped carbons
    rdkit_reaction = AllChem.ReactionFromSmarts(mapped_reaction_smiles, useSmiles=True)
    CarbonR, CarbonP = 0,0
    unmappedR, unmappedP = 0,0
    BondsR, BondsP = 0,0
    unmappedBondsR, unmappedBondsP = 0,0
    C0C0BondsR, C0C0BondsP = 0,0

    # Counts carbons, unmapped carbons, and different types of bonds in reactants
    for reactant in rdkit_reaction.GetReactants():
      for atomR in reactant.GetAtoms():
         if atomR.GetSymbol() == 'C':
          CarbonR += 1
          if atomR.GetAtomMapNum() == 0:
            unmappedR+=1
          for neiR in atomR.GetNeighbors():
            if neiR.GetSymbol() == 'C':
              BondsR+=0.5
              if neiR.GetAtomMapNum() == 0 and atomR.GetAtomMapNum() == 0:
                C0C0BondsR+=0.5
              if neiR.GetAtomMapNum() == 0 or atomR.GetAtomMapNum() == 0:
                unmappedBondsR+=0.5

    # Counts carbons, unmapped carbons, and different types of bonds in products
    for product in rdkit_reaction.GetProducts():
      for atomP in product.GetAtoms():
         if atomP.GetSymbol() == 'C':
          CarbonP += 1
          if atomP.GetAtomMapNum() == 0:
            unmappedP+=1
          for neiP in atomP.GetNeighbors():
            if neiP.GetSymbol() == 'C':
              BondsP+=0.5
              if neiP.GetAtomMapNum() == 0 and atomP.GetAtomMapNum() == 0:
                C0C0BondsP+=0.5
              if neiP.GetAtomMapNum() == 0 or atomP.GetAtomMapNum() == 0:
                unmappedBondsP+=0.5

    # Put in negative cases for unclean data
    TotalBonds = BondsR+BondsP
    if TotalBonds == 0:
      TotalBonds = -1
    propUnmapped = -1
    if CarbonR+CarbonP > 0:
        propUnmapped = (unmappedR+unmappedP)/(CarbonR+CarbonP)

    unmappedCarbons = unmappedR+unmappedP
    propTotalUnmappedBonds = (unmappedBondsR + unmappedBondsP)/(TotalBonds)
    propMapUnmapBonds = (unmappedBondsR-C0C0BondsR+unmappedBondsP-C0C0BondsP)/(TotalBonds)
    propC0C0Bonds = (C0C0BondsR+C0C0BondsP)/(TotalBonds)

    UnmappedMetrics = [TotalBonds,propUnmapped,propTotalUnmappedBonds,propMapUnmapBonds,propC0C0Bonds,unmappedCarbons]

    # Number of bonds that are different between reactants and products, number of atoms with same neighbors in reactants and products
    Rlist = []
    Plist = []
    SameNeighborCarbons, SameNeighborCarbonsWithoutUnmapped = [],[]
    DiffNeighborCarbons, lostCarbons = [],[]
    NumberDiffBondsX2, diffNeighbors, sameNeighbors, totalBondsX2, NumberLostCarbons = 0,0,0,0,0
    sameNeighborsWithoutUnmapped = 0

    # Iterate through mapped carbons in reactants
    for reactant in rdkit_reaction.GetReactants():
      for atomR in reactant.GetAtoms():
         if atomR.GetSymbol() == 'C':
            atomRsymbol = atomR.GetSymbol()
            if f'{atomRsymbol}{atomR.GetAtomMapNum()}' in product_labels.keys():
              if atomR.GetAtomMapNum() != 0:
               # Iterate through neighboring carbons for a carbon in reactants and adds them to a list 
               for neiR in atomR.GetNeighbors():
                   if neiR.GetSymbol() == 'C': Rlist.append(neiR.GetAtomMapNum())
               # Iterate through the carbons in products
               for product in rdkit_reaction.GetProducts():
                   for atomP in product.GetAtoms():
                    if atomP.GetSymbol() == 'C':

                      # Compare reactant neighbor and product neighbor:
                      # Continues only if we are looking at the same mapped atom in reactants and products
                      if atomP.GetAtomMapNum() != 0:
                        if f'{atomRsymbol}{atomR.GetAtomMapNum()}' == f'{atomP.GetSymbol()}{atomP.GetAtomMapNum()}':
                          # Iterates through neighboring atoms in products and adds them to a list              
                          for neiP in atomP.GetNeighbors():
                            if neiP.GetSymbol() == 'C': Plist.append(neiP.GetAtomMapNum())

                          # Counts bonds that differ between reactants and products by using lists of neighbors
                          # Number of different bonds includes if a carbon has added or lost bonds
                          # Several conditions present to account for unmapped carbons since they are not double counted
                          for x in Rlist:
                                  if x not in Plist and x != 0:
                                        NumberDiffBondsX2 += 1
                                  elif x not in Plist and x == 0:
                                        NumberDiffBondsX2 += 2
                          for x in Plist:
                                  if x not in Rlist and x != 0:
                                        NumberDiffBondsX2 += 1
                                  elif x not in Rlist and x == 0:
                                        NumberDiffBondsX2 += 2

                          # Assess if atoms have retained neighbors between reactant and product
                          # Different neighbors includes if a carbon has added or lost neighbors
                          if all(element in Plist for element in Rlist) and all(element in Rlist for element in Plist):
                                sameNeighbors+=1
                                Rlist = []
                                Plist = []
                          else:
                                diffNeighbors+=1
                                Rlist = []
                                Plist = []

            # Counts bonds to lost carbons (carbons that are in reactants but not in products)
            else:
              for neiR in atomR.GetNeighbors():
                  if neiR.GetSymbol() == 'C' and atomR.GetAtomMapNum() != 0:
                    if neiR.GetAtomMapNum() == 0:
                      NumberDiffBondsX2 += 2
                    else:
                      NumberDiffBondsX2 += 1

    propDiffBonds = [NumberDiffBondsX2/2/TotalBonds]
    propSameNeighbors = [sameNeighbors*2/TotalCarbons]
    magicList = propDiffBonds + propSameNeighbors + CarbonMetrics + UnmappedMetrics
    return magicList

In [ ]:
def generate_metrics(df):
    rm_prop_diff_bonds = []
    rm_prop_carbon_conserved = []
    rm_total_num_c = []
    rm_prop_gained = []
    rm_prop_lost = []
    rm_total_bonds = []
    rm_prop_unmapped = []
    rm_prop_unmapped_bonds = []
    rm_prop_map_unmap = []
    rm_prop_unmap_unmap = []
    rm_total_unmapped_c = []

    for i in range(df.shape[0]):
        try:
            rxnmapper_map_smiles = df["rxnmapper_mapped_smiles"].iloc[i]

            carbonMetricsList = carbonMetrics(rxnmapper_map_smiles)
            rm_prop_diff_bonds.append(carbonMetricsList[0])
            rm_prop_carbon_conserved.append(carbonMetricsList[1])
            rm_total_num_c.append(carbonMetricsList[2])
            rm_prop_gained.append(carbonMetricsList[3])
            rm_prop_lost.append(carbonMetricsList[4])
            rm_total_bonds.append(carbonMetricsList[5])
            rm_prop_unmapped.append(carbonMetricsList[6])
            rm_prop_unmapped_bonds.append(carbonMetricsList[7])
            rm_prop_map_unmap.append(carbonMetricsList[8])
            rm_prop_unmap_unmap.append(carbonMetricsList[9])
            rm_total_unmapped_c.append(carbonMetricsList[10])

        except BaseException as e:
            print(e)
            print('rxnmapper error')
            print(rxnmapper_map_smiles)
            rm_prop_carbon_conserved.append(-1)
            rm_prop_gained.append(-1)
            rm_prop_lost.append(-1)
            rm_prop_diff_bonds.append(-1)
            rm_total_bonds.append(-1)
            rm_prop_unmapped.append(-1)
            rm_prop_unmapped_bonds.append(-1)
            rm_prop_map_unmap.append(-1)
            rm_prop_unmap_unmap.append(-1)
            rm_total_num_c.append(-1)
            rm_total_unmapped_c.append(-1)


    df["rm_prop_c_conserved_neighbors"] = rm_prop_carbon_conserved
    df["rm_prop_gained_c"] = rm_prop_gained
    df["rm_prop_lost_c"] = rm_prop_lost
    df["rm_prop_unconserved_c_bonds"] = rm_prop_diff_bonds
    df["rm_prop_unmapped_c"] = rm_prop_unmapped
    df["rm_prop_unmapped_c_bonds"] = rm_prop_unmapped_bonds
    df["rm_prop_map_to_unmap_c_bonds"] = rm_prop_map_unmap
    df["rm_prop_unmap_to_unmap_c_bonds"] = rm_prop_unmap_unmap
    df["rm_total_c"] = rm_total_num_c
    df["rm_total_unmap_c"] = rm_total_unmapped_c
    df["rm_total_bonds"] = rm_total_bonds

    return 

generate_metrics(gpt)
gpt = gpt[gpt['rm_prop_unmapped_c'] != -1]

In [ ]:
def substructure_mapping_all(reactants_smiles, products_smiles, radius=2, nBits=2048):
    scores_dict = {"MF_sim": 0, "APF_sim": 0, "MACCSKF_sim": 0}
    
    # Morgan fingerprints
    reactants_fp = [AllChem.GetMorganFingerprint(Chem.MolFromSmiles(smiles), radius) for smiles in reactants_smiles]
    products_fp = [AllChem.GetMorganFingerprint(Chem.MolFromSmiles(smiles), radius) for smiles in products_smiles]

    fp = []
    for i, product_fp in enumerate(products_fp):
        for j, reactant_fp in enumerate(reactants_fp):
            similarity = DataStructs.TanimotoSimilarity(product_fp, reactant_fp)
            fp.append(similarity)
        filtered = [x for x in fp if x < 0.95]
        scores_dict['MF_sim'] = sum(filtered) / len(filtered) if filtered else 0
    
    # Atom-pair fingerprints
    reactants_fp = [AllChem.GetAtomPairFingerprint(Chem.MolFromSmiles(smiles)) for smiles in reactants_smiles]
    products_fp = [AllChem.GetAtomPairFingerprint(Chem.MolFromSmiles(smiles)) for smiles in products_smiles]
    
    fp = []
    for i, product_fp in enumerate(products_fp):
        for j, reactant_fp in enumerate(reactants_fp):
            similarity = DataStructs.TanimotoSimilarity(product_fp, reactant_fp)
            fp.append(similarity)
    filtered = [x for x in fp if x < 0.95]
    scores_dict['APF_sim'] = sum(filtered) / len(filtered) if filtered else 0

    # Molecular ACCess System (MACCS) Keys
    reactants_fp = [AllChem.GetMACCSKeysFingerprint(Chem.MolFromSmiles(smiles)) for smiles in reactants_smiles]
    products_fp = [AllChem.GetMACCSKeysFingerprint(Chem.MolFromSmiles(smiles)) for smiles in products_smiles]
    
    fp = []
    for i, product_fp in enumerate(products_fp):
        for j, reactant_fp in enumerate(reactants_fp):
            similarity = DataStructs.TanimotoSimilarity(product_fp, reactant_fp)
            fp.append(similarity)
    filtered = [x for x in fp if x < 0.95]
    scores_dict['MACCSKF_sim'] = sum(filtered) / len(filtered) if filtered else 0

    return scores_dict

gpt['MorganFingerprint_similarity'] = [list() for x in range(len(gpt.index))]
gpt['AtomPairFingerprint_similarity'] = [list() for x in range(len(gpt.index))]
gpt['MACCSKeysFingerprint_similarity'] = [list() for x in range(len(gpt.index))]

for index,row in gpt.iterrows():
    substrate = row['substrates_converted_smiles']
    pdt = row['products_converted_smiles']

    scores = substructure_mapping_all(substrate, pdt)
    gpt.at[index, 'MorganFingerprint_similarity'] = scores['MF_sim']
    gpt.at[index, 'AtomPairFingerprint_similarity'] = scores['APF_sim']
    gpt.at[index, 'MACCSKeysFingerprint_similarity'] = scores['MACCSKF_sim']

In [ ]:
test_with_atommap = gpt.drop(['substrates', 'products', 'substrates_converted_smiles', 'products_converted_smiles', 'rxnmapper_mapped_smiles', 'MorganFingerprint_similarity', 'AtomPairFingerprint_similarity', 'MACCSKeysFingerprint_similarity'], axis=1)
mapper = 'rm'
columns = [
       f'{mapper}_prop_c_conserved_neighbors',
       f'{mapper}_prop_gained_c', 
       f'{mapper}_prop_lost_c',
       f'{mapper}_prop_unconserved_c_bonds',
       f'{mapper}_prop_unmapped_c',
       f'{mapper}_prop_unmapped_c_bonds', 
       f'{mapper}_prop_map_to_unmap_c_bonds', 
       f'{mapper}_prop_unmap_to_unmap_c_bonds', 
       f'{mapper}_total_c', 
       f'{mapper}_total_unmap_c', 
       f'{mapper}_total_bonds', 
       'substrates_converted_smiles', 
       'products_converted_smiles', 
       'rxnmapper_mapped_smiles',
       'substrates', 'products']

test_with_fingerprint = gpt.drop(columns, axis=1)

In [ ]:
# test_with_fingerprint.drop(['validations'], axis=1, inplace=True)
X_test = test_with_fingerprint.to_numpy()
predictions = rfc_for_fingerprint.predict(X_test)
test_with_fingerprint['validations'] = predictions
gpt['validations_with_fingerprint'] = predictions

counts = test_with_fingerprint['validations'].value_counts().sort_index()

plt.figure(figsize=(6, 4))
plt.bar(counts.index.astype(str), counts.values, color='skyblue', edgecolor='black')
plt.xlabel('Predicted Class')
plt.ylabel('Count')
plt.title('Distribution of Predicted Validations with Fingerprint Scores')
plt.tight_layout()
plt.show()

# test_with_atommap.drop(['validations'], axis=1, inplace=True)
X_test = test_with_atommap.to_numpy()
predictions = rfc_for_atommap.predict(X_test)
test_with_atommap['validations'] = predictions
gpt['validations_with_atommap'] = predictions

counts = test_with_atommap['validations'].value_counts().sort_index()

plt.figure(figsize=(6, 4))
plt.bar(counts.index.astype(str), counts.values, color='skyblue', edgecolor='black')
plt.xlabel('Predicted Class')
plt.ylabel('Count')
plt.title('Distribution of Predicted Validations with Atom Map Metrics')
plt.tight_layout()
plt.show()

In [ ]:
flagged = gpt[
    (gpt['validations_with_atommap'] == 0) &
    (gpt['validations_with_fingerprint'] == 0)
]

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=2570873b-d089-4ed7-b836-6c2df496af15' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>